In [81]:
import yfinance as yf
import pandas as pd
import numpy as np
import trading_clock as tc
from data import *
from statistics import NormalDist


In [82]:
SPXdata = SecurityData("^spx", security_type="ETF")

start = tc.get_today() - timedelta(days=2)
end = tc.get_today()
SPXdata.get_yfinance_data(start, end, "1m", name="yesterdata")

yesterdays_returns = np.log(
    SPXdata["yesterdata"][['Close']] / SPXdata["yesterdata"][['Close']].shift()).dropna()["Close"].to_numpy()


In [83]:
yesterdays_mean = np.mean(yesterdays_returns)
yesterdays_std = np.std(yesterdays_returns)
# print(yesterdays_mean*120)

current_price = 4443.25
desired_price = 4448.51
time_to_sell = 360
confidence = 0.4

upper_pct_change = NormalDist(
    mu=time_to_sell*yesterdays_mean, 
    sigma=yesterdays_std*time_to_sell**0.5
).inv_cdf(confidence+(1-confidence)/2)

lower_pct_change = NormalDist(
    mu=time_to_sell*yesterdays_mean,
    sigma=yesterdays_std*time_to_sell**0.5
).inv_cdf((1-confidence)/2)

print(f"{confidence*100}% Confidence Interval")
upper_range = (1+upper_pct_change) * current_price
print(f"Upper Range After {time_to_sell} minutes: ${upper_range:0.2f}")
lower_range = (1+lower_pct_change) * current_price
print(f"Lower Range After {time_to_sell} minutes: ${lower_range:0.2f}")

if np.log(desired_price/current_price) > time_to_sell*yesterdays_mean:
    chance_of_exceeding_range = 1-(NormalDist(
        mu=time_to_sell*yesterdays_mean,
        sigma=yesterdays_std*time_to_sell**0.5
        ).cdf(np.log(desired_price/current_price)))
else:
    chance_of_exceeding_range = (NormalDist(
        mu=time_to_sell*yesterdays_mean,
        sigma=yesterdays_std*time_to_sell**0.5
    ).cdf(np.log(desired_price/current_price)))

print(f"Chance of exceeding ${desired_price} after {time_to_sell} minutes: {chance_of_exceeding_range*100:0.2f}%")


40.0% Confidence Interval
Upper Range After 360 minutes: $4491.45
Lower Range After 360 minutes: $4458.92
Chance of exceeding $4448.51 after 360 minutes: 80.52%
